In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

from shutil import which
import time

import os
import pandas as pd
import numpy as np
import re


# from selenium import webdriver

import requests
from lxml.html import fromstring
import json
import os
import re



In [40]:
def scroll(t):
    height = driver.execute_script("return document.body.scrollHeight")
    for scrol in range(100,height-1000,200):
        driver.execute_script(f"window.scrollTo(0,{scrol})")
        time.sleep(t)

In [41]:
def extract(page_source, start_sep, end_sep):
    source=page_source
#     start_sep='DIMENSIONS'
#     end_sep='MATERIAL'
    result=[]
    tmp=source.split(start_sep)
    for par in tmp:
        if end_sep in par:
            result.append(par.split(end_sep)[0])
#     result = result[3].replace('\\n', '\n')
#     result = "".join([s for s in result.strip().splitlines(True) if s.strip()])
    return result

def extract2(string, start, end):
    result = re.search('%s(.*)%s' % (start, end), string).group(1)
    return result

In [54]:
def get_info():
    price = driver.find_element(By.XPATH, "//div[@class = 'title size-m text-price price float-left ng-binding']")
    product_details['price'] = float(price.text[1:].replace(',', ''))

    sku = driver.find_element(By.XPATH, "/html[1]/body[1]/div[4]/div[1]/div[3]/div[1]/div[1]/form[1]/div[2]/ul[2]/li[2]/span[1]")
    product_details['sku'] = sku.text

    avaliable_new = driver.find_element(By.XPATH, "/html[1]/body[1]/div[4]/div[1]/div[3]/div[1]/div[1]/form[1]/div[2]/ul[2]/li[3]/span[1]")
    product_details['avaliable_new'] = avaliable_new.text

    shipping_charge = driver.find_element(By.XPATH, "/html[1]/body[1]/div[4]/div[1]/div[3]/div[1]/div[1]/form[1]/div[2]/ul[2]/li[4]/span[1]")
    product_details['shipping_charge'] = float(shipping_charge.text[1:].replace(',', ''))

In [55]:
def empty_item():
    product_details = {
        'name': '',
        'sku': '',
        'gtin': '',
        'color': '',
        'finish': '',
        'price': '',
        'product_description': '',
        'technical_description': '',
        'display_dimensions': '',
        'length': '',
        'width': '',
        'height': '',
        'depth': '',
        'diameter': '',
        'weight': '',
        'arm_height': '',
        'seat_height': '',
        'table_clearance_height': '',
        'maximum_dimensions': '',
        'hero_image': '',
        'variant_image': '',
        'images': '',
        'files': '',
        'installations': '',
        'designer': '',
        'collection': '',
        'categories': '',
        'available_new': '',
        'made_in': '',
        'voltage': '',
        'bulb_type': '',
        'bulb_text': '',
        'canopy_dimensions': '',
        'ul_listed': '',
        'certifications': '',
        'warranty_&_care': '',
        'position': '',
        'notes': '',
        'url':  ''
    }
    return product_details




In [111]:
chrome_options = Options()


if False:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')

prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
chrome_path = r"C:\Users\chaitanya\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
global driver
driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
driver.get("https://shop.horm.it/en/casamania-horm/products")

time.sleep(1)

cookies_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,  "//button[@class = 'button primary large expanded']")))
cookies_path.click()

items = []

upholstry_list = ['BASIC | Nuance Basic', 'TOP | Nuance TOP', 'EXCLUSIVE | Nuance Exclusive', 'MAJOR | Nuance Major Leather']

products = driver.find_elements(By.XPATH, "//div[@id = 'productsContent']/div/div")
product_no = 0
for product in products:
    product_no += 1
    items_single = []
    product_details = {}
    product_url = product.find_element(By.XPATH, ".//div[@class = 'cont-image']/a[@href]")
    
    href = product_url.get_attribute("href")
    product_details['url'] = href
    
    hero_image_path = product.find_element(By.XPATH, ".//div[@class = 'cont-image']/a/picture/img")
    product_details['hero_image'] = hero_image_path.get_attribute('src').replace('/h_', '/b_' )
    
    name_path = product.find_element(By.XPATH, ".//div[@class = 'cont-info']/h3/a")
    product_details['name'] = name_path.text
    
    category_path = product.find_element(By.XPATH, ".//div[@class = 'cont-info']/p[1]")
    product_details['categories'] = category_path.text
    
    driver.execute_script("window.open('" +href +"');")
    # switch to new window
    driver.switch_to.window(driver.window_handles[1])
    
#     response_product = requests.get(href)
#     page_source_product = response_product.text
#     info = json.loads(extract(page_source_product, '<script type="application/ld+json">', ' </script>')[-1])
      
    scroll(0.5)
    
    infos = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "/html[1]/body[1]/div[4]/div[1]/div[3]/div[2]/div[1]/div[1]/div[1]/div[1]/p[1]")))
    desc = []
    
    for info in infos:
        desc.append(info.text)
        
    product_details['product description'] = '\n'.join(desc)
    try:
        designer = driver.find_element(By.XPATH, "/html[1]/body[1]/div[4]/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/div[5]/b[1]")
        product_details['designer'] = designer.text
    except:
        product_details['designer'] = None
        
    try:
        collection = driver.find_element(By.XPATH, "/html[1]/body[1]/div[4]/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/div[4]/b[1]")
        product_details['collection'] = collection.text
    except:
        product_details['collection'] = None
        
    installations = []
    installations_path =  driver.find_elements(By.XPATH, "//img[@class = 'lazy']")
    for inst in installations_path:
        installations.append(inst.get_attribute('src'))
    product_details['installations'] = '\n'.join(installations)
    
    files = []
    files_path = driver.find_elements(By.XPATH, "//div[@class = 'grid-x grid-margin-x grid-padding-x small-up-2 medium-up-4']/div/a")
    for file in files_path:
        file_name = file.get_attribute('title')
        file_content = file.get_attribute('href')
        details = file_name + ': ' + file_content
        files.append(details)
    product_details['files'] = '\n'.join(files)
    
        
    images = []
    images_path = driver.find_elements(By.XPATH, "//img[@class = 'nav-carousel-image flickity-lazyloaded']")
    for img in images_path:
        images.append(img.get_attribute('src'))
    product_details['images'] = '\n'.join(images)
        
    time.sleep(3)
        
    try:
        options = driver.find_elements(By.XPATH, "//ul[@id = 'articles-menu-container']/li")
    except:
        options = []
    if len(options) < 1:
        
        get_info()
        items_single.append(product_details.copy())
        items.append(product_details.copy())
    else:
        options_1 = driver.find_elements(By.XPATH, "//div[@class = 'options']")[0].find_elements(By.XPATH, ".//div[@class = 'option ng-scope']")
        for opt_1 in options_1:
            option_name1 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[0].text
            
            driver.find_elements(By.XPATH, "//div[@class = 'options']")[0].click()
            opt_1.click()
            
            more_options = driver.find_elements(By.XPATH, "//div[@class = 'options']")
            
            if len(more_options) == 1:
            
                time.sleep(1)
#                 option_name1 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[0].text
                opt1_value = opt_1.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                try:
                    opt1_image = opt_1.find_element(By.XPATH, ".//label/img").get_attribute('src')
                    product_details[f'option({option_name1})'] = opt1_value + ': ' + opt1_image
                except:
                    product_details[f'option({option_name1})'] = opt1_value
                    

                get_info()
                items_single.append(product_details.copy())
                items.append(product_details.copy())
                print(product_details)
            else:
                options_2 = driver.find_elements(By.XPATH, "//div[@class = 'options']")[1].find_elements(By.XPATH, ".//div[@class = 'option ng-scope']")
                for opt_2 in options_2:
                    
                    driver.find_elements(By.XPATH, "//div[@class = 'options']")[1].click()
                    opt_2.click()
                    more_options = driver.find_elements(By.XPATH, "//div[@class = 'options']")
                    time.sleep(1)
                    if len(more_options) == 2:
                        
                        option_name1 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[0].text
                        opt1_value = opt_1.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                        try:
                            opt1_image = opt_1.find_element(By.XPATH, ".//label/img").get_attribute('src')
                            product_details[f'option({option_name1})'] = opt1_value + ': ' + opt1_image
                        except:
                            product_details[f'option({option_name1})'] = opt1_value
                            
                        option_name2 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[1].text
                        opt2_value = opt_2.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                        try:
                            opt2_image = opt_2.find_element(By.XPATH, ".//label/img").get_attribute('src')
                            product_details[f'option({option_name2})'] = opt2_value + ': ' + opt2_image
                        except:
                            product_details[f'option({option_name2})']  = opt2_value

                        get_info()
                        items_single.append(product_details.copy())
                        items.append(product_details.copy())
                        print(product_details)
                        
                    else:
                        options_3 = driver.find_elements(By.XPATH, "//div[@class = 'options']")[2].find_elements(By.XPATH, ".//div[@class = 'option ng-scope']")
                        for opt_3 in options_3:
                            driver.find_elements(By.XPATH, "//div[@class = 'options']")[2].click()
                            opt_3.click()
                            more_options = driver.find_elements(By.XPATH, "//div[@class = 'options']")
                            time.sleep(1)
                            if len(more_options) == 3:
                                
                                option_name1 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[0].text
                                opt1_value = opt_1.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                try:
                                    opt1_image = opt_1.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                    product_details[f'option({option_name1})'] = opt1_value + ': ' + opt1_image
                                except:
                                    product_details[f'option({option_name1})'] = opt1_value

                                option_name2 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[1].text
                                opt2_value = opt_2.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                try:
                                    opt2_image = opt_2.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                    product_details[f'option({option_name2})'] = opt2_value + ': ' + opt2_image
                                except:
                                    product_details[f'option({option_name2})']  = opt2_value

                                option_name3 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[2].text
                                opt3_value = opt_3.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                try:
                                    opt3_image = opt_3.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                    product_details[f'option({option_name3})'] = opt3_value + ': ' + opt3_image
                                except:
                                    product_details[f'option({option_name3})']  = opt3_value

                                get_info()
                                items_single.append(product_details.copy())
                                items.append(product_details.copy())
                                print(product_details)
                            
                            else:
                                options_4 = driver.find_elements(By.XPATH, "//div[@class = 'options']")[3].find_elements(By.XPATH, ".//div[@class = 'option ng-scope']")
                                for opt_4 in options_4:
                                    driver.find_elements(By.XPATH, "//div[@class = 'options']")[3].click()
                                    opt_4.click()
                                    more_options = driver.find_elements(By.XPATH, "//div[@class = 'options']")
                                    time.sleep(1)
                                    if len(more_options) == 4:

                                        option_name1 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[0].text
                                        opt1_value = opt_1.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                        try:
                                            opt1_image = opt_1.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                            product_details[f'option({option_name1})'] = opt1_value + ': ' + opt1_image
                                        except:
                                            product_details[f'option({option_name1})'] = opt1_value

                                        option_name2 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[1].text
                                        opt2_value = opt_2.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                        try:
                                            opt2_image = opt_2.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                            product_details[f'option({option_name2})'] = opt2_value + ': ' + opt2_image
                                        except:
                                            product_details[f'option({option_name2})']  = opt2_value

                                        option_name3 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[2].text
                                        opt3_value = opt_3.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                        try:
                                            opt3_image = opt_3.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                            product_details[f'option({option_name3})'] = opt3_value + ': ' + opt3_image
                                        except:
                                            product_details[f'option({option_name3})']  = opt3_value
                                            
                                        option_name4 = driver.find_elements(By.XPATH, "//div[@class = 'options']/div[@class = 'option']/label")[3].text
                                        opt4_value = opt_4.find_element(By.XPATH, ".//label/span[@class = 'ng-binding ng-scope']").text
                                        try:
                                            opt4_image = opt_4.find_element(By.XPATH, ".//label/img").get_attribute('src')
                                            product_details[f'option({option_name4})'] = opt4_value + ': ' + opt4_image
                                        except:
                                            product_details[f'option({option_name4})']  = opt4_value
                                        

                                        get_info()
                                        items_single.append(product_details.copy())
                                        items.append(product_details.copy())
                                        print(product_details)
    
    
    for prod in items_single[1:]:
        prod['hero_image'] = None    
            
    
    df = pd.DataFrame(items_single, )
    df.to_excel(f'data/{product_no}.xlsx')
    
    driver.close()
    #back to main window
    driver.switch_to.window(driver.window_handles[0])



C:\Users\CHAITA~1\AppData\Local\Temp/ipykernel_35832/2685449053.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


{'url': 'https://shop.horm.it/en/products/polyethylene-chair-her-opaca_497277', 'hero_image': 'https://img.edilportale.com/product-thumbs/b_her-opaca-horm-italia-497277-rel481930f6.jpg', 'name': 'HER OPACA', 'categories': 'Polyethylene chair', 'product description': 'Her is a Chair in dyed-through matt polyethylene.\n\n“Him & Her are born directly from the ideas behind the Panton Chair. An evolution of the hermaphroditic original, declined into the harmony of the two sexes. They assume sculpted forms like naked models of seduction but they are not ashamed.” Cit. Fabio Novembre\nChairs crafted from rotationally mass-dyed moulded polyethylene. Their shape derives from the three-dimensional scanning of two plaster sculptures.', 'designer': 'Fabio Novembre', 'collection': 'Him & Her', 'installations': 'https://img.edilportale.com/products/HER-OPACA-Casamania-Horm-497277-dim6a850371.jpg', 'files': 'Assembly, Maintenance, Warranty: https://img.edilportale.com/catalogs/Assembly-Maintenance-Wa

{'url': 'https://shop.horm.it/en/products/polyethylene-chair-him-opaca_497278', 'hero_image': 'https://img.edilportale.com/product-thumbs/b_HIM-OPACA-Casamania-Horm-497278-rel16e5ec14.jpg', 'name': 'HIM OPACA', 'categories': 'Polyethylene chair', 'product description': 'Him is a Chair in dyed-through matt polyethylene.\n\n“Him & Her are born directly from the ideas behind the Panton Chair. An evolution of the hermaphroditic original, declined into the harmony of the two sexes. They assume sculpted forms like naked models of seduction but they are not ashamed.” Cit. Fabio Novembre\nChairs crafted from rotationally mass-dyed moulded polyethylene. Their shape derives from the three-dimensional scanning of two plaster sculptures.', 'designer': 'Fabio Novembre', 'collection': 'Him & Her', 'installations': 'https://img.edilportale.com/products/HIM-OPACA-Casamania-Horm-497278-dima8dac236.jpg', 'files': 'Assembly, Maintenance, Warranty: https://img.edilportale.com/catalogs/Assembly-Maintenance

{'url': 'https://shop.horm.it/en/products/polyethylene-coat-stand-alberto-coat-rack_494949', 'hero_image': 'https://img.edilportale.com/product-thumbs/b_alberto-coat-rack-horm-italia-494949-rel1f98bee6.jpg', 'name': 'ALBERTO | Coat rack', 'categories': 'Polyethylene coat stand', 'product description': 'Coat stand in mass dyed polyethylene made by rotational moulding.\nKnobs in Natural oak wood.', 'designer': 'Fabrica', 'collection': 'Alberto', 'installations': 'https://img.edilportale.com/products/ALBERTO-Coat-rack-Casamania-Horm-494949-dimccb8da0d.jpg', 'files': 'Assembly, Maintenance, Warranty: https://img.edilportale.com/catalogs/Assembly-Maintenance-Warranty-Casamania-Horm-494949-cate3c0d754.pdf\nTechnical Data Sheet: https://img.edilportale.com/catalogs/Technical-Data-Sheet-Casamania-Horm-494949-cat3c257599.pdf\nWood and veneers: https://img.edilportale.com/catalogs/Wood-and-veneers-Casamania-Horm-494949-cat7b1436de.pdf\nMatt polyethylene: https://img.edilportale.com/catalogs/Matt

{'url': 'https://shop.horm.it/en/products/2-seater-fabric-sofa-summit-standard_497928', 'hero_image': 'https://img.edilportale.com/product-thumbs/b_summit-standard-horm-italia-497928-relec774b17.jpg', 'name': 'SUMMIT STANDARD', 'categories': '2 seater fabric sofa', 'product description': 'Summit Standard is a 2 seater fabric sofa.\nStructure composed of black painted metal frame with mdf bottoms. Upholstered in leather, faux leather or fabric.', 'designer': 'Giulio Iacchetti', 'collection': 'Summit', 'installations': 'https://img.edilportale.com/products/SUMMIT-STANDARD-Casamania-Horm-497928-dim36ae8766.jpg\nhttps://img.edilportale.com/products/SUMMIT-STANDARD-Casamania-Horm-497928-dimf2e85ad9.jpg', 'files': 'Technical Data Sheet: https://img.edilportale.com/catalogs/Technical-Data-Sheet-Casamania-Horm-497928-catfec87058.pdf\nFabrics "Nuance" Collection: https://img.edilportale.com/catalogs/Fabrics-Nuance-Collection-Casamania-Horm-497403-cat49443801.pdf\nContract Fabrics by Kvadrat & V

{'url': 'https://shop.horm.it/en/products/2-seater-fabric-sofa-summit-standard_497928', 'hero_image': 'https://img.edilportale.com/product-thumbs/b_summit-standard-horm-italia-497928-relec774b17.jpg', 'name': 'SUMMIT STANDARD', 'categories': '2 seater fabric sofa', 'product description': 'Summit Standard is a 2 seater fabric sofa.\nStructure composed of black painted metal frame with mdf bottoms. Upholstered in leather, faux leather or fabric.', 'designer': 'Giulio Iacchetti', 'collection': 'Summit', 'installations': 'https://img.edilportale.com/products/SUMMIT-STANDARD-Casamania-Horm-497928-dim36ae8766.jpg\nhttps://img.edilportale.com/products/SUMMIT-STANDARD-Casamania-Horm-497928-dimf2e85ad9.jpg', 'files': 'Technical Data Sheet: https://img.edilportale.com/catalogs/Technical-Data-Sheet-Casamania-Horm-497928-catfec87058.pdf\nFabrics "Nuance" Collection: https://img.edilportale.com/catalogs/Fabrics-Nuance-Collection-Casamania-Horm-497403-cat49443801.pdf\nContract Fabrics by Kvadrat & V

WebDriverException: Message: chrome not reachable
  (Session info: chrome=105.0.5195.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x0032DF13+2219795]
	Ordinal0 [0x002C2841+1779777]
	Ordinal0 [0x001D4100+803072]
	Ordinal0 [0x001C9BF2+760818]
	Ordinal0 [0x001BCD35+707893]
	Ordinal0 [0x001C1122+725282]
	Ordinal0 [0x001C5699+743065]
	Ordinal0 [0x001D5610+808464]
	Ordinal0 [0x0022E93A+1173818]
	Ordinal0 [0x0021E616+1107478]
	Ordinal0 [0x001F7F89+950153]
	Ordinal0 [0x001F8F56+954198]
	GetHandleVerifier [0x00622CB2+3040210]
	GetHandleVerifier [0x00612BB4+2974420]
	GetHandleVerifier [0x003C6A0A+565546]
	GetHandleVerifier [0x003C5680+560544]
	Ordinal0 [0x002C9A5C+1808988]
	Ordinal0 [0x002CE3A8+1827752]
	Ordinal0 [0x002CE495+1827989]
	Ordinal0 [0x002D80A4+1867940]
	BaseThreadInitThunk [0x76C06739+25]
	RtlGetFullPathName_UEx [0x770B90AF+1215]
	RtlGetFullPathName_UEx [0x770B907D+1165]


In [120]:
chrome_options = Options()


if False:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')

prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
chrome_path = r"C:\Users\kchai\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
global driver
driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
driver.get("https://shop.horm.it/en/casamania-horm/products/1")

time.sleep(1)

cookies_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,  "//button[@class = 'button primary large expanded']")))
cookies_path.click()

items = []

upholstry_list = ['BASIC | Nuance Basic', 'TOP | Nuance TOP', 'EXCLUSIVE | Nuance Exclusive', 'MAJOR | Nuance Major Leather']
# upholstry_list = ['Nuance Basic', 'TOP | Nuance TOP', 'EXCLUSIVE | Nuance Exclusive', 'MAJOR | Nuance Major Leather']

# upholstry_list = [i.lower() for i in upholstry_list]
products = driver.find_elements(By.XPATH, "//div[@id = 'productsContent']/div/div")

for product in products:
    product_details = {}
    product_url = product.find_element(By.XPATH, ".//div[@class = 'cont-image']/a[@href]")
    
    href = product_url.get_attribute("href")
    product_details['url'] = href
    
    name_path = product.find_element(By.XPATH, ".//div[@class = 'cont-info']/h3/a")
    product_details['name'] = name_path.text
    
    name_extension_path = product.find_element(By.XPATH, ".//div[@class = 'cont-info']/p[1]")
    product_details['name_extension'] = name_extension_path.text
    
    driver.execute_script("window.open('" +href +"');")
    # switch to new window
    driver.switch_to.window(driver.window_handles[1])


    category_path = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//ul[@class = 'breadcrumbs']/li/a")))[-1]
    product_details['category'] = category_path.text
        
    print(product_details)
    items.append(product_details)
  
    
    
    
    driver.close()
    #back to main window
    driver.switch_to.window(driver.window_handles[0])



C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\2941425603.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


{'url': 'https://shop.horm.it/en/products/polyethylene-chair-her-opaca_497277', 'name': 'HER OPACA', 'name_extension': 'Polyethylene chair', 'category': 'Garden chairs'}
{'url': 'https://shop.horm.it/en/products/polyethylene-chair-him-opaca_497278', 'name': 'HIM OPACA', 'name_extension': 'Polyethylene chair', 'category': 'Garden chairs'}
{'url': 'https://shop.horm.it/en/products/polyethylene-coat-stand-alberto-coat-rack_494949', 'name': 'ALBERTO | Coat rack', 'name_extension': 'Polyethylene coat stand', 'category': 'Coat racks'}
{'url': 'https://shop.horm.it/en/products/2-seater-fabric-sofa-summit-standard_497928', 'name': 'SUMMIT STANDARD', 'name_extension': '2 seater fabric sofa', 'category': 'Sofas'}
{'url': 'https://shop.horm.it/en/products/upholstered-fabric-easy-chair-cherish-lounge_495792', 'name': 'CHERISH LOUNGE', 'name_extension': 'Upholstered fabric easy chair', 'category': 'Armchairs'}
{'url': 'https://shop.horm.it/en/products/lacquered-mdf-sideboard-leon-decor-on-the-base_

{'url': 'https://shop.horm.it/en/products/square-garden-table-t1-outdoor-bistrot_498327', 'name': 'T1 OUTDOOR BISTROT', 'name_extension': 'Square garden table', 'category': 'Garden tables'}
{'url': 'https://shop.horm.it/en/products/square-drop-leaf-table-t4-bistrot-square-table_498333', 'name': 'T4 BISTROT | Square table', 'name_extension': 'Square drop-leaf table', 'category': 'Tables'}
{'url': 'https://shop.horm.it/en/products/square-fenix-ntm-table-tango-bistrot_498344', 'name': 'TANGO BISTROT', 'name_extension': 'Square Fenix-NTM® table', 'category': 'Tables'}
{'url': 'https://shop.horm.it/en/products/contemporary-style-rectangular-fenix-ntm-table-tango-fisso_498345', 'name': 'TANGO FISSO', 'name_extension': 'Contemporary style rectangular Fenix-NTM® table', 'category': 'Tables'}
{'url': 'https://shop.horm.it/en/products/rug-manin_497696', 'name': 'MANIN', 'name_extension': 'Rug', 'category': 'Rugs'}
{'url': 'https://shop.horm.it/en/products/rug-gritti_495956', 'name': 'GRITTI', 'n

In [56]:
len(items)

246

In [57]:
import pandas as pd
import numpy as np

df_cat = pd.DataFrame(items, )
df_cat


,url,name,name_extension,category
0,https://shop.horm.it/en/products/polyethylene-...,HER OPACA,Polyethylene chair,Garden chairs
1,https://shop.horm.it/en/products/polyethylene-...,HIM OPACA,Polyethylene chair,Garden chairs
2,https://shop.horm.it/en/products/polyethylene-...,ALBERTO | Coat rack,Polyethylene coat stand,Coat racks
3,https://shop.horm.it/en/products/2-seater-fabr...,SUMMIT STANDARD,2 seater fabric sofa,Sofas
4,https://shop.horm.it/en/products/upholstered-f...,CHERISH LOUNGE,Upholstered fabric easy chair,Armchairs
...,...,...,...,...
241,https://shop.horm.it/en/products/wooden-sidebo...,LEON WOOD ON-THE-BASE IMBIANCATO,Wooden sideboard with doors,Sideboards
242,https://shop.horm.it/en/products/wooden-sidebo...,LEON WOOD MOCA,Wooden sideboard with doors,Sideboards
243,https://shop.horm.it/en/products/wooden-sidebo...,LEON WOOD BOIS BLOND,Wooden sideboard with doors,Sideboards
244,https://shop.horm.it/en/products/wooden-sidebo...,LEON WOOD IMBIANCATO,Wooden sideboard with doors,Sideboards


In [144]:
# COLEMAN | Sofa
for filename in os.listdir('data/')[197: 198]:
    if '.xlsx' in filename:
        data = pd.read_excel(f'data/{filename}')
        if data['name'][0] == 'COLEMAN | Sofa':
            print(filename)

56.xlsx


In [111]:
import os
import numpy as np
import pandas as pd

upholstry_list = ['Nuance Basic', 'Nuance TOP', 'Nuance Exclusive', 'Nuance Major Leather', \
                  'Basic Nuance', 'Top Nuance', 'Exclusive Nuance', 'Leather Nuance Major']

# df_all = pd.DataFrame()
count = 0 
one_list = []
for filename in os.listdir('data/')[197: 198]:
    if '.xlsx' in filename:
        data = pd.read_excel(f'data/{filename}')
        data = data.replace({np.nan: None})

        df_new = pd.DataFrame()
        
        opt_columns = [col for col in data.columns if 'option' in col.lower()]
        for i in range(len(data)):
            if data['images'][0] is None:
                data['images'] = data['hero_image'][0]
            try:
                data['images'][i] = data['images'][i].replace('g_', 'b_')
            except:
                print(i)
            for opt in opt_columns:
                try:
                    data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
                except:
                    pass
                
        data['variant_image'] = None
        for i in range(len(data)):
            data['variant_image'][i] = data['images'][i].split('\n')[0]
        

        
        uph_colms  = [col for col in data.columns if 'upholstery' in col.lower()]
        if len(uph_colms) > 0:
            for i in range(len(data)):
                for col in uph_colms:
                    for uphol in upholstry_list:
                        try:
                            if uphol.lower() in data[col][i].lower():
                                df_new = df_new.append(data.iloc[i], ignore_index = True)
                        except:
                            df_new = df_new.append(data.iloc[i], ignore_index = True)
            if len(df_new) == 0:
                df_new = df_new.append(data, ignore_index = True)
                print(filename)
                print(f'check the file {filename}')
        else:
            df_new = df_new.append(data, ignore_index = True)
            
            
        df_all = df_all.append(df_new, ignore_index = True)
        count += 1
        
df_all

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['variant_image'][i] = data['images'][i].split('\n')[0]
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(data.iloc[i], ignore_index = True)
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df_new, ignore_index = True)
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['variant_image'][i] = data['images'][i].split('\n')[0]
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(data, ignore_index = True)
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df_new, ignore_index = True)
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy 

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

...

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

93.xlsx
check the file 93.xlsx


C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3459626045.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,...,option(Legs and shell finishing),option(Ends finishing),option(Product),option(Doors finishing),option(Modul - Size),option(Top/extensions finishing),option(Trays and castors finish),option(Top / Base finish),option(Interior layout),option(Shelf finishing)
0,0,https://shop.horm.it/en/products/polyethylene-...,https://img.edilportale.com/product-thumbs/b_h...,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,https://shop.horm.it/en/products/low-rectangul...,https://img.edilportale.com/product-thumbs/b_r...,RAGTIME | Tavolino rettangolare basso,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,https://img.edilportale.com/products/RAGTIME-T...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112504,940,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112505,941,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112506,942,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112507,943,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
count

48

In [113]:
os.listdir('data/')[242]

'98.xlsx'

In [146]:
import pandas as pd
import numpy as np
import re

upholstry_list = ['Nuance Basic', 'Nuance TOP', 'Nuance Exclusive', 'Nuance Major Leather', \
                  'Basic Nuance', 'Top Nuance', 'Exclusive Nuance', 'Leather Nuance Major']

data = pd.read_excel(f'data/54.xlsx')
data = data.replace({np.nan: None})
df_new = pd.DataFrame()
opt_columns = [col for col in data.columns if 'option' in col.lower()]
for i in range(len(data)):
    if data['images'][0] is None:
        data['images'] = data['hero_image'][0]
    try:
        data['images'][i] = data['images'][i].replace('g_', 'b_')
    except:
        print(i)
    for opt in opt_columns:
        data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
        
        

uph_colms  = [col for col in data.columns if 'upholstery' in col.lower()]
if len(uph_colms) > 0:
    for i in range(len(data)):
        for col in uph_colms:
            for uphol in upholstry_list:
                if uphol.lower() in data[col][i].lower():
                    df_new = df_new.append(data.iloc[i], ignore_index = True)
    if len(df_new) == 0:
        df_new = df_new.append(data, ignore_index = True)
else:
    df_new = df_new.append(data, ignore_index = True)
    
df_new

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\2631757338.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['images'][i] = data['images'][i].replace('g_', 'b_')
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\2631757338.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[opt][i] = re.sub('\(.*?\)', '', data[opt][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\2631757338.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(data.iloc[i], ignore_index = True)


,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,images,option(Model - Size),option(Structure finishing),option(Upholstery category),option(Colour),price,sku,avaliable_new,shipping_charge
0,11,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,https://img.edilportale.com/product-thumbs/b_C...,COLEMAN A | 90 x 240 cm,Brick aluminium: https://img.edilportale.com/m...,Basic Nuance fabric: https://img.edilportale.c...,Alfa T-101 34 : https://img.edilportale.com/ma...,3500.11,COLAAA-T0,Ready to ship in 28 working days,1735.69
1,12,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,https://img.edilportale.com/product-thumbs/b_C...,COLEMAN A | 90 x 240 cm,Brick aluminium: https://img.edilportale.com/m...,Basic Nuance fabric: https://img.edilportale.c...,Makkiano L1542 01 : https://img.edilportale.co...,3685.84,COLAAA-T0,Ready to ship in 28 working days,1735.69
2,13,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,https://img.edilportale.com/product-thumbs/b_C...,COLEMAN A | 90 x 240 cm,Brick aluminium: https://img.edilportale.com/m...,Basic Nuance fabric: https://img.edilportale.c...,Makkiano L1542 02 : https://img.edilportale.co...,3685.84,COLAAA-T0,Ready to ship in 28 working days,1735.69
3,14,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,https://img.edilportale.com/product-thumbs/b_C...,COLEMAN A | 90 x 240 cm,Brick aluminium: https://img.edilportale.com/m...,Basic Nuance fabric: https://img.edilportale.c...,Makkiano L1542 03 : https://img.edilportale.co...,3685.84,COLAAA-T0,Ready to ship in 28 working days,1735.69
4,15,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,https://img.edilportale.com/product-thumbs/b_C...,COLEMAN A | 90 x 240 cm,Brick aluminium: https://img.edilportale.com/m...,Basic Nuance fabric: https://img.edilportale.c...,Makkiano L1542 05 : https://img.edilportale.co...,3685.84,COLAAA-T0,Ready to ship in 28 working days,1735.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2532,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,https://img.edilportale.com/product-thumbs/b_C...,COLEMAN AA | 105 x 240 cm,Polished aluminium: https://img.edilportale.co...,Leather Nuance Major: https://img.edilportale....,Nuvola 179 Lino : https://img.edilportale.com/...,6596.21,COLAAAL-P1,Ready to ship in 28 working days,2017.16
1049,2533,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palo

In [114]:
opt_columns = [col for col in df_all.columns if 'option' in col.lower()]
opt_columns

['option(Colour)',
 'option(Upholstery)',
 'option(Model)',
 'option(External finish)',
 'option(Structure finishing)',
 'option(Upholstery category)',
 'option(Top finishing)',
 'option(Dimensions)',
 'option(Glass deco)',
 'option(Cable colour)',
 'option(Finishing)',
 'option(Legs finishing)',
 'option(Upholstery colour)',
 'option(Upholstery type)',
 'option(Options)',
 'option(Shell finishing)',
 'option(Shelves)',
 'option(Model – Dimensions (cm))',
 'option(Structure colour)',
 'option(Seat finish)',
 'option(Top dimensions (cm))',
 'option(Table top finishing)',
 'option(Under-chair colour)',
 'option(Base)',
 'option(Structure/top finishing)',
 'option(Feet finishing)',
 'option(Composition)',
 'option(Shelves finishing)',
 'option(Armrest upholstery)',
 'option(Model - Size)',
 'option(Lampshade upholstery)',
 'option(Plug adaptor)',
 'option(Bed frame: Model and Dimension)',
 'option(Headboard: Model and Dimension)',
 'option(Panels finishing)',
 'option(Sled base finishing)

In [115]:
df_all

,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,...,option(Legs and shell finishing),option(Ends finishing),option(Product),option(Doors finishing),option(Modul - Size),option(Top/extensions finishing),option(Trays and castors finish),option(Top / Base finish),option(Interior layout),option(Shelf finishing)
0,0,https://shop.horm.it/en/products/polyethylene-...,https://img.edilportale.com/product-thumbs/b_h...,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,https://shop.horm.it/en/products/low-rectangul...,https://img.edilportale.com/product-thumbs/b_r...,RAGTIME | Tavolino rettangolare basso,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,https://img.edilportale.com/products/RAGTIME-T...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112504,940,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112505,941,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112506,942,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112507,943,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df_all = df_all.reindex(columns=[*df_all.columns.tolist(), 'GTIN',\
                               'finish', 'technical_description', 'color', 'gtin',  \
                    'diameter', 'arm height', 'table clearance height', \
                    'maximum dimensions','made_in', 'canopy dimensions','length', \
                                 'width', 'height', 'depth', 'weight', 'seat_height',\
                              'voltage', 'bulb_type', 'bulb_text', 'UL Listed', \
                    'certifications', 'warranty & care', 'position', 'notes'], fill_value= None)

In [117]:
df_all['variant_image'] = None
for i in range(len(df_all)):
    df_all['variant_image'][i] = df_all['images'][i].split('\n')[0]
df_all

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\3988413024.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['variant_image'][i] = df_all['images'][i].split('\n')[0]


,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,...,weight,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,0,https://shop.horm.it/en/products/polyethylene-...,https://img.edilportale.com/product-thumbs/b_h...,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,https://shop.horm.it/en/products/low-rectangul...,https://img.edilportale.com/product-thumbs/b_r...,RAGTIME | Tavolino rettangolare basso,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,https://img.edilportale.com/products/RAGTIME-T...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112504,940,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112505,941,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112506,942,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112507,943,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
len(df_all['url'].unique())

239

In [119]:
df_all['name'].unique()

array(['HER OPACA', 'RAGTIME | Tavolino rettangolare basso',
       'TOUT LE JOUR | Armchair', 'HOO', 'KORZINA',
       'AUTOREGGENTE | Coffee table', 'KANT', 'KANT LIFT', 'CARNABY',
       'TOUT LE JOUR | Coffee table', 'ZONTIK', 'LEPEL SMOOTH',
       'RAGTIME | High square coffee table', 'GRANNY', 'KOUCH',
       'LEPEL QUILTED', 'OUCH', 'TRIBECA', 'KI', 'MARITIME WOOD',
       'MARITIME PADDED', 'RAY | Easy chair',
       'RAGTIME | Tavolino rettangolare alto', 'LA-DINA', 'LOBBY',
       'QUADROR™ 01', 'MEDUSE | High side table', 'QUADROR™ 02',
       'MEDUSE | Low coffee table', 'MEDUSE | Marble table',
       'MEDUSE | Glass table', 'PABLITA', 'REMEMBER ME CHAIR',
       'HEXAGON PIETRA', 'MORFINO | Bean bag', 'MARIA',
       'MORFINO | Square pouf', 'ROBOX', 'QUADROR™ 03',
       'STEREO STACKABLE POLYPROPYLENE', 'STEREO WOOD POLYPROPYLENE',
       'NIÑA PINTA & SANTA MARIA', 'MILADY', 'QUADROR™ 04', 'LEON BOX',
       'QUADROR™ | Sculpture', 'PRESSIOUS', 'MA.RE',
       'ALBINO

In [17]:
missing = [9,  95(SIENNA BICOLOR), 97(SPECTRO 5 DOORS), 238(SPECTRO 3 DOORS ) , 160(TWEE T), 176, 10]

SyntaxError: invalid syntax (3987390185.py, line 1)

In [121]:
df_all = df_all.drop_duplicates(keep='first' , ignore_index=True)
df_all

,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,...,weight,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,0,https://shop.horm.it/en/products/polyethylene-...,https://img.edilportale.com/product-thumbs/b_h...,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,https://shop.horm.it/en/products/low-rectangul...,https://img.edilportale.com/product-thumbs/b_r...,RAGTIME | Tavolino rettangolare basso,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,https://img.edilportale.com/products/RAGTIME-T...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111987,940,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111988,941,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111989,942,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111990,943,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
df_all.iloc[83435]

Unnamed: 0                                                       703
url                https://shop.horm.it/en/products/lacquered-mdf...
hero_image                                                      None
name                                          LEON DECOR ON-THE-BASE
categories                                   Lacquered MDF sideboard
                                         ...                        
UL Listed                                                        NaN
certifications                                                   NaN
warranty & care                                                  NaN
position                                                         NaN
notes                                                            NaN
Name: 83435, Length: 87, dtype: object

In [125]:
df_cat = df_cat[['url', 'category']]
df_cat

,url,category
0,https://shop.horm.it/en/products/polyethylene-...,Garden chairs
1,https://shop.horm.it/en/products/polyethylene-...,Garden chairs
2,https://shop.horm.it/en/products/polyethylene-...,Coat racks
3,https://shop.horm.it/en/products/2-seater-fabr...,Sofas
4,https://shop.horm.it/en/products/upholstered-f...,Armchairs
...,...,...
241,https://shop.horm.it/en/products/wooden-sidebo...,Sideboards
242,https://shop.horm.it/en/products/wooden-sidebo...,Sideboards
243,https://shop.horm.it/en/products/wooden-sidebo...,Sideboards
244,https://shop.horm.it/en/products/wooden-sidebo...,Sideboards


In [126]:
df_full = pd.merge(df_all, df_cat,  how ='inner', left_on = 'url', right_on = 'url')
df_full

,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,...,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes,category
0,0,https://shop.horm.it/en/products/polyethylene-...,https://img.edilportale.com/product-thumbs/b_h...,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
1,1,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
2,2,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
3,3,https://shop.horm.it/en/products/polyethylene-...,None,HER OPACA,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,https://img.edilportale.com/products/HER-OPACA...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
4,0,https://shop.horm.it/en/products/low-rectangul...,https://img.edilportale.com/product-thumbs/b_r...,RAGTIME | Tavolino rettangolare basso,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,https://img.edilportale.com/products/RAGTIME-T...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coffee tables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111987,940,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
111988,941,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
111989,942,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs
111990,943,https://shop.horm.it/en/products/aluminium-cha...,None,RAPHIA ALLUMINIO,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,https://img.edilportale.com/products/RAPHIA-AL...,"Assembly, Maintenance, Warranty: https://img.e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garden chairs


In [127]:
df_full['new_name'] = None
for i in range(len(df_full)):
    df_full['new_name'][i] = df_full['name'][i] + '- ' + df_full['categories'][i]

C:\Users\kchai\AppData\Local\Temp\ipykernel_22936\1207103905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['new_name'][i] = df_full['name'][i] + '- ' + df_full['categories'][i]


In [128]:
df_full['new_name']

0                             HER OPACA- Polyethylene chair
1                             HER OPACA- Polyethylene chair
2                             HER OPACA- Polyethylene chair
3                             HER OPACA- Polyethylene chair
4         RAGTIME | Tavolino rettangolare basso- Low rec...
                                ...                        
111987      RAPHIA ALLUMINIO- Aluminium chair with armrests
111988      RAPHIA ALLUMINIO- Aluminium chair with armrests
111989      RAPHIA ALLUMINIO- Aluminium chair with armrests
111990      RAPHIA ALLUMINIO- Aluminium chair with armrests
111991      RAPHIA ALLUMINIO- Aluminium chair with armrests
Name: new_name, Length: 111992, dtype: object

In [6]:
all_columns = df.columns.to_list()
all_columns

['name',
 'sku',
 'hero_image',
 'category',
 'new_name',
 'url',
 'categories',
 'product description',
 'designer',
 'collection',
 'installations',
 'files',
 'images',
 'option(Colour)',
 'price',
 'avaliable_new',
 'shipping_charge',
 'variant_image',
 'option(Upholstery)',
 'option(Model)',
 'option(External finish)',
 'option(Structure finishing)',
 'option(Upholstery category)',
 'option(Top finishing)',
 'option(Dimensions)',
 'option(Glass deco)',
 'option(Cable colour)',
 'option(Finishing)',
 'option(Legs finishing)',
 'option(Upholstery colour)',
 'option(Upholstery type)',
 'option(Options)',
 'option(Shell finishing)',
 'option(Shelves)',
 'option(Model – Dimensions (cm))',
 'option(Structure colour)',
 'option(Seat finish)',
 'option(Top dimensions (cm))',
 'option(Table top finishing)',
 'option(Under-chair colour)',
 'option(Base)',
 'option(Structure/top finishing)',
 'option(Feet finishing)',
 'option(Composition)',
 'option(Shelves finishing)',
 'option(Armrest uph

In [92]:
# all_columns.remove('price')
all_columns[10:15]

['images', 'option(Colour)', 'sku', 'avaliable_new', 'shipping_charge']

In [7]:
all_columns = df.columns.to_list()
opt_columns = [col for col in all_columns if 'option' in col]
opt_columns

['option(Colour)',
 'option(Upholstery)',
 'option(Model)',
 'option(External finish)',
 'option(Structure finishing)',
 'option(Upholstery category)',
 'option(Top finishing)',
 'option(Dimensions)',
 'option(Glass deco)',
 'option(Cable colour)',
 'option(Finishing)',
 'option(Legs finishing)',
 'option(Upholstery colour)',
 'option(Upholstery type)',
 'option(Options)',
 'option(Shell finishing)',
 'option(Shelves)',
 'option(Model – Dimensions (cm))',
 'option(Structure colour)',
 'option(Seat finish)',
 'option(Top dimensions (cm))',
 'option(Table top finishing)',
 'option(Under-chair colour)',
 'option(Base)',
 'option(Structure/top finishing)',
 'option(Feet finishing)',
 'option(Composition)',
 'option(Shelves finishing)',
 'option(Armrest upholstery)',
 'option(Model - Size)',
 'option(Lampshade upholstery)',
 'option(Plug adaptor)',
 'option(Bed frame: Model and Dimension)',
 'option(Headboard: Model and Dimension)',
 'option(Panels finishing)',
 'option(Sled base finishing)

In [141]:
df_all[df_all['name'] == 'COLEMAN | Sofa']['price']

69743    3500.11
69744    3685.84
69745    3685.84
69746    3685.84
69747    3685.84
          ...   
71845    6831.92
71846    6831.92
71847    6831.92
71848    6831.92
71849    6831.92
Name: price, Length: 2106, dtype: object

In [174]:
df_col = df_full[df_full['new_name'] == 'COLEMAN | Sofa- 2 seater sofa with removable cover']
df_col

,Unnamed: 0,url,hero_image,name,categories,product description,designer,collection,installations,files,...,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes,category,new_name
83433,11,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
83434,12,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
83435,13,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
83436,14,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
83437,15,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85534,2532,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
85535,2533,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
85536,2534,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...
85537,2535,https://shop.horm.it/en/products/2-seater-sofa...,None,COLEMAN | Sofa,2 seater sofa with removable cover,"Coleman is a fabric sofa, also available in le...",Ludovica + Roberto Palomba,Coleman,https://img.edilportale.com/products/COLEMAN-S...,Technical Data Sheet: https://img.edilportale....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sofas,COLEMAN | Sofa- 2 seater sofa with removable c...


In [8]:
opt_columns.append('name')

In [179]:
df_col.drop_duplicates(subset = opt_columns+ ['name'], keep='first' , ignore_index=True)['name']

0       COLEMAN | Sofa
1       COLEMAN | Sofa
2       COLEMAN | Sofa
3       COLEMAN | Sofa
4       COLEMAN | Sofa
             ...      
1048    COLEMAN | Sofa
1049    COLEMAN | Sofa
1050    COLEMAN | Sofa
1051    COLEMAN | Sofa
1052    COLEMAN | Sofa
Name: name, Length: 1053, dtype: object

In [11]:
df = pd.read_excel('files/horm.xlsx')
df

,name,sku,hero_image,category,new_name,url,categories,product description,designer,collection,...,weight,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,HER OPACA,CM8636-PCBI,https://img.edilportale.com/product-thumbs/b_h...,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HER OPACA,CM8636-PCNE,NaN,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HER OPACA,CM8636-PCGB,NaN,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HER OPACA,CM8636-PCGP,NaN,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RAGTIME | Tavolino rettangolare basso,RT120060,https://img.edilportale.com/product-thumbs/b_r...,Coffee tables,RAGTIME | Tavolino rettangolare basso- Low rec...,https://shop.horm.it/en/products/low-rectangul...,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97114,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97115,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97116,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97117,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
names = list(df['name'].unique())
names

['HER OPACA',
 'RAGTIME | Tavolino rettangolare basso',
 'TOUT LE JOUR | Armchair',
 'HOO',
 'KORZINA',
 'AUTOREGGENTE | Coffee table',
 'KANT',
 'KANT LIFT',
 'CARNABY',
 'TOUT LE JOUR | Coffee table',
 'ZONTIK',
 'LEPEL SMOOTH',
 'RAGTIME | High square coffee table',
 'GRANNY',
 'KOUCH',
 'LEPEL QUILTED',
 'OUCH',
 'TRIBECA',
 'KI',
 'MARITIME WOOD',
 'MARITIME PADDED',
 'RAY | Easy chair',
 'RAGTIME | Tavolino rettangolare alto',
 'LA-DINA',
 'LOBBY',
 'QUADROR™ 01',
 'MEDUSE | High side table',
 'QUADROR™ 02',
 'MEDUSE | Low coffee table',
 'MEDUSE | Marble table',
 'MEDUSE | Glass table',
 'PABLITA',
 'REMEMBER ME CHAIR',
 'HEXAGON PIETRA',
 'MORFINO | Bean bag',
 'MARIA',
 'MORFINO | Square pouf',
 'ROBOX',
 'QUADROR™ 03',
 'STEREO STACKABLE POLYPROPYLENE',
 'STEREO WOOD POLYPROPYLENE',
 'NIÑA PINTA & SANTA MARIA',
 'MILADY',
 'QUADROR™ 04',
 'LEON BOX',
 'QUADROR™ | Sculpture',
 'PRESSIOUS',
 'MA.RE',
 'ALBINO FAMILY | Wooden coffee table',
 'WEB STOPPER',
 'REMEMBER ME BISTROT'

In [13]:
df_err = pd.DataFrame()
for name in names:
    data = df[df['name'] == name]
    data = data.drop_duplicates(subset = opt_columns, keep='first' , ignore_index=True)
    df_err = df_err.append(data, ignore_index = True)
df_err

C:\Users\kchai\AppData\Local\Temp\ipykernel_15784\983571464.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_err = df_err.append(data, ignore_index = True)


,name,sku,hero_image,category,new_name,url,categories,product description,designer,collection,...,weight,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,HER OPACA,CM8636-PCBI,https://img.edilportale.com/product-thumbs/b_h...,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HER OPACA,CM8636-PCNE,NaN,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HER OPACA,CM8636-PCGB,NaN,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HER OPACA,CM8636-PCGP,NaN,Garden chairs,HER OPACA- Polyethylene chair,https://shop.horm.it/en/products/polyethylene-...,Polyethylene chair,Her is a Chair in dyed-through matt polyethyle...,Fabio Novembre,Him & Her,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RAGTIME | Tavolino rettangolare basso,RT120060,https://img.edilportale.com/product-thumbs/b_r...,Coffee tables,RAGTIME | Tavolino rettangolare basso- Low rec...,https://shop.horm.it/en/products/low-rectangul...,Low rectangular marble coffee table,Ragtime is part of the accessories designed by...,Ludovica + Roberto Palomba,Ragtime,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97114,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97115,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97116,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97117,RAPHIA ALLUMINIO,"CM8732-R9016,CM8735T3_NUANCE EXCLUSIVE",NaN,Garden chairs,RAPHIA ALLUMINIO- Aluminium chair with armrests,https://shop.horm.it/en/products/aluminium-cha...,Aluminium chair with armrests,Raphia Alluminio is the offspring of a researc...,LucidiPevere,Raphia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
df_err.iloc[83433]

name                                          MILES | Sectional sofa
sku                                                     MILHHSXAA-T2
hero_image                                                       NaN
category                                                       Sofas
new_name           MILES | Sectional sofa- Sectional sofa with re...
                                         ...                        
UL Listed                                                        NaN
certifications                                                   NaN
warranty & care                                                  NaN
position                                                         NaN
notes                                                            NaN
Name: 83433, Length: 88, dtype: object

In [188]:
df = df.drop_duplicates(subset = opt_columns+['name'], keep='first' , ignore_index=True)


In [69]:
df_full.to_csv('horm.csv')

In [379]:
df.to_excel('files/horm.xlsx', index=False)

In [204]:
df[df['name'] == names[8]]['price'].iloc[0]

2387.64

In [364]:
for i in df.iloc[93334]:
    print(i)

GIRO | Coffee table
CM9956-T0+ CM9946PRORN_NUANCE BASIC
nan
Coffee tables
GIRO | Coffee table- Round coffee table
https://shop.horm.it/en/products/round-coffee-table-giro-coffee-table_405745
Round coffee table
nan
nan
Claesson Koivisto Rune
https://img.edilportale.com/products/GIRO-Coffee-table-Casamania-Horm-405745-dimf69112a4.jpg
Technical Data Sheet: https://img.edilportale.com/catalogs/Technical-Data-Sheet-Casamania-Horm-405745-cat96334b8a.pdf
Assembly, Maintenance, Warranty: https://img.edilportale.com/catalogs/Assembly-Maintenance-Warranty-Casamania-Horm-405745-catb2f77ea3.pdf
Fabrics "Nuance" Collection: https://img.edilportale.com/catalogs/Fabrics-Nuance-Collection-Casamania-Horm-405745-cat49e83473.pdf
Contract Fabrics by Kvadrat & Vescom: https://img.edilportale.com/catalogs/Contrac-Fabrics-by-Kvadrat-Vescom-Casamania-Horm-405745-cata37c9513.pdf
Faux Leather "Nuance" Collection: https://img.edilportale.com/catalogs/Faux-Leather-Nuance-Collection-Casamania-Horm-405745-cat36a0ce

In [377]:
num = 233
print(df[df['name'] == names[num]]['url'].iloc[0])
df[df['name'] == names[num]][['name', 'price']]

https://shop.horm.it/en/products/rattan-chair-with-armrests-raphia-rattan_106490


,name,price
96453,RAPHIA RATTAN,657.84
96454,RAPHIA RATTAN,778.17
96455,RAPHIA RATTAN,778.17
96456,RAPHIA RATTAN,778.17
96457,RAPHIA RATTAN,778.17
...,...,...
96881,RAPHIA RATTAN,909.54
96882,RAPHIA RATTAN,909.54
96883,RAPHIA RATTAN,909.54
96884,RAPHIA RATTAN,909.54


In [378]:
count = 0
for name in names:

    data = df[df['name'] == name]
    if len(data)>1:
        if data['price'].iloc[0] != data['price'].iloc[1]:
            print(f'Check the product {name}')
            print(data['price'].iloc[0], data['price'].iloc[1])
            print(count)
            print('*'*10)
        else:
            pass
    count += 1

Check the product TOUT LE JOUR | Coffee table
1943.41 2332.49
9
**********
Check the product MEDUSE | Marble table
2370.17 2525.38
29
**********
Check the product MARIA
211.28 102.14
35
**********
Check the product QUADROR™ 03
7185.62 7935.62
38
**********
Check the product QUADROR™ 04
14193.99 13142.59
43
**********
Check the product LEON BOX
1825.69 1290.52
44
**********
Check the product PRESSIOUS
449.6 457.61
46
**********
Check the product RIDDLED | Table
4672.25 5446.29
51
**********
Check the product T1 BISTROT | Round table
443.59 496.66
56
**********
Check the product T1 BISTROT | Square table
443.59 496.66
57
**********
Check the product TWINE | Low coffee table
366.49 436.58
58
**********
Check the product RAPHIA ROCKING
1178.58 1298.74
61
**********
Check the product SENDAI
5006.7 6309.44
65
**********
Check the product CARLOS WHITE LACQUERED
2835.79 2637.53
67
**********
Check the product RIDDLED FRONT
4153.56 4499.02
69
**********
Check the product IO E TE
762.02 662.89
8

In [31]:
df = pd.read_excel(r'C:\Users\kchai\Downloads\horm.xlsx')
df

,name,sku,hero_image,categories,new_name,url,categories_sub,product description,designer,collection,...,weight,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,93-08,KX30113R-L_Nero RAL9011,https://img.edilportale.com/product-thumbs/b_p...,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93-08,KX30113R-L_Bianco RAL9016,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93-08,KX30113R-L_Grigio RAL7037,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93-08,KX30113R-L_Grigio beige RAL7006,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,93-08,KX30113R-L_Rosso vino RAL3005,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97114,ZONTIK,CM8880L-R9011,https://img.edilportale.com/product-thumbs/b_Z...,Umbrella stands,ZONTIK- Polyethylene umbrella stand,https://shop.horm.it/en/products/polyethylene-...,Polyethylene umbrella stand,The Zontik umbrella stand keeps things well or...,Karim Rashid,Zontik & Korzina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97115,ZONTIK,CM8880L-R7037,NaN,Umbrella stands,ZONTIK- Polyethylene umbrella stand,https://shop.horm.it/en/products/polyethylene-...,Polyethylene umbrella stand,The Zontik umbrella stand keeps things well or...,Karim Rashid,Zontik & Korzina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97116,ZONTIK,CM8880L-R7006,NaN,Umbrella stands,ZONTIK- Polyethylene umbrella stand,https://shop.horm.it/en/products/polyethylene-...,Polyethylene umbrella stand,The Zontik umbrella stand keeps things well or...,Karim Rashid,Zontik & Korzina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97117,ZONTIK,CM8880L-R3005,NaN,Umbrella stands,ZONTIK- Polyethylene umbrella stand,https://shop.horm.it/en/products/polyethylene-...,Polyethylene umbrella stand,The Zontik umbrella stand keeps things well or...,Karim Rashid,Zontik & Korzina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.iloc[8275]['option(Model)']

'4 sides module: L 180 P 80 H 39: https://img.edilportale.com/matt-thumbs/matt_fd484e02-992f-40b6-b20a-f18ff5263627.png'

In [33]:
all_columns = df.columns.to_list()
opt_columns = [col for col in all_columns if 'option' in col]
opt_columns

['option(Color)',
 'option(Upholstery)',
 'option(Model)',
 'option(External finish)',
 'option(Structure finishing)',
 'option(Upholstery category)',
 'option(Top finishing)',
 'option(Dimensions)',
 'option(Glass deco)',
 'option(Cable colour)',
 'option(Finishing)',
 'option(Legs finishing)',
 'option(Upholstery colour)',
 'option(Upholstery type)',
 'option(Options)',
 'option(Shell finishing)',
 'option(Shelves)',
 'option(Model Dimensions in cm)',
 'option(Structure colour)',
 'option(Seat finish)',
 'option(Top dimensions in cm)',
 'option(Table top finishing)',
 'option(Under chair colour)',
 'option(Base)',
 'option(Structure/top finishing)',
 'option(Feet finishing)',
 'option(Composition)',
 'option(Shelves finishing)',
 'option(Armrest upholstery)',
 'option(Model Size)',
 'option(Lampshade upholstery)',
 'option(Plug adaptor)',
 'option(Bed frame Model and Dimension)',
 'option(Headboard Model and Dimension)',
 'option(Panels finishing)',
 'option(Sled base finishing)',
 '

In [15]:
df[df.duplicated(opt_columns )]['sku']

128                                CM8890-PCBI
129                                CM8890-PCBG
435                              CM8937-VCNE,/
436      CM8937-VCNE,CM8937C-S-T0-NUANCE BASIC
437      CM8937-VCNE,CM8937C-S-T0-NUANCE BASIC
                         ...                  
97114                            CM8880L-R9011
97115                            CM8880L-R7037
97116                            CM8880L-R7006
97117                            CM8880L-R3005
97118                              CM8880-PCBI
Name: sku, Length: 9488, dtype: object

In [8]:
df.drop_duplicates(subset = opt_columns , keep='first' , ignore_index=True)


,name,sku,hero_image,categories,new_name,url,categories_sub,product description,designer,collection,...,weight,seat_height,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,93-08,KX30113R-L_Nero RAL9011,https://img.edilportale.com/product-thumbs/b_p...,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93-08,KX30113R-L_Bianco RAL9016,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93-08,KX30113R-L_Grigio RAL7037,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93-08,KX30113R-L_Grigio beige RAL7006,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,93-08,KX30113R-L_Rosso vino RAL3005,NaN,Bookcases,'93-'08- Modular wooden bookcase,https://shop.horm.it/en/products/modular-woode...,Modular wooden bookcase,The ‘93-’08 bookcase features a solid wood str...,NaN,Carlo Cumini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87626,WORN | Armchair,"CM9602-HPCZ_NUANCE MAJOR,CM9601_BRACCIOLI ECOP...",NaN,Armchairs,WORN | Armchair- Armchair with armrests,https://shop.horm.it/en/products/armchair-with...,Armchair with armrests,NaN,Samuel Wilkinson,Worn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87627,WOW! PLUS,ZW140BL,https://img.edilportale.com/product-thumbs/b_p...,Tables,WOW! PLUS- Extending rectangular table,https://shop.horm.it/en/products/extending-rec...,Extending rectangular table,"A table of simple and elegant design, also sui...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87628,WOW! PLUS,ZW140BR,NaN,Tables,WOW! PLUS- Extending rectangular table,https://shop.horm.it/en/products/extending-rec...,Extending rectangular table,"A table of simple and elegant design, also sui...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87629,WOW! PLUS,ZW140ZZ,NaN,Tables,WOW! PLUS- Extending rectangular table,https://shop.horm.it/en/products/extending-rec...,Extending rectangular table,"A table of simple and elegant design, also sui...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
test = df['option(Model)'][8275]
temp = test.split(':')
add = ':'.join(temp[-2:])
name = ''.join(temp[:-2])
test = name + ': ' + add
test

'4 sides module L 180 P 80 H 39:  https://img.edilportale.com/matt-thumbs/matt_fd484e02-992f-40b6-b20a-f18ff5263627.png'

In [45]:
from collections import Counter

for i in range(len(df)):
    for opt in opt_columns:
        try:
            count = Counter(df[opt][i])
            if count[':'] >2:
#                 temp = df[opt][i].split(':')
#                 add = ':'.join(temp[-2:])
#                 name = ''.join(temp[:-2])
#                 df[opt][i] = name + ': ' + add
                print(i, opt)
        except:
            pass
            

In [46]:
df.to_excel('files/horm.xlsx')